In [0]:
import sys

sys.path.append("../include_utils/")

#from IPython.parallel import Client
import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, STDOUT, check_output, call
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
import pysam
from collections import OrderedDict, namedtuple
import operator
import multiprocessing as mp
import glob

In [0]:
cd "~/eckertlab/Mitra/mapping/split_parallel/collapsed/work"

In [0]:
bam_dir = "."
analysis_dir = os.path.join(bam_dir, "samtools1.3")
if not os.path.exists(analysis_dir):
    os.makedirs(analysis_dir)
assert os.path.exists(analysis_dir)

In [0]:
bam_files = !ls *.bam

In [0]:
len(bam_files) == 381

In [0]:
samtools = "/home/cfriedline/bin/samtools"
bcftools = "/home/cfriedline/bin/bcftools"
picard = "/home/cfriedline/gpfs/src/broadinstitute-picard-03a1d72/dist/picard.jar"
java = "/home/cfriedline/g/src/jdk1.8.0_92/bin/java"
perl = "/home/cfriedline/gpfs/opt/ActivePerl-5.18/bin/perl"

In [0]:
assembly = "/gpfs_fs/home/eckertlab/SugarPine_genome/pila.v1.0.scafSeq_mapped.fasta"

In [0]:
!$samtools faidx {assembly}

In [0]:
def create_ploidy_file(args):
    import os
    bam_files, analysis_dir = args
    ploidy_file = os.path.join(analysis_dir, "%s.ploidy" % "all")
    if not os.path.exists(ploidy_file):
        with open(ploidy_file, "w") as o:
            for b in bam_files:
                name = "%s" % os.path.basename(b).split("_")[0]
                ploidy = 2
                o.write("%s\t%d\n" % (name, ploidy))
    return ploidy_file

In [0]:
def create_split_beds(nodes, bed):
    lines = 0
    for line in open(bed):
        lines += 1
    print(lines, lines//nodes)
    per_bed = lines//nodes
    cmd = "split -a 3 -d -l %d %s contig.bed." % (per_bed, bed)
    call(cmd.split())
create_split_beds(150, "../contigs.bed")

In [0]:
beds = !ls contig.bed.*

In [0]:
rc = u.get_client(profile="sge")
dv, lv = u.get_views(rc)
len(dv)

In [0]:
with dv.sync_imports():
    from subprocess import Popen, PIPE, STDOUT, check_output, call
    import os, sys, socket, glob

In [0]:
def create_parallel_bams(args):
    os.chdir("/home/cfriedline/eckertlab/Mitra/mapping/split_parallel/collapsed/work/")
    samtools, bam_file, bed_file = args
    num = bed_file.split(".")[-1]
    out = "%s.%s" % (bam_file, num)
    if not os.path.exists(out):
        cmd = "%s view -L %s -b %s -o %s" % (samtools, bed_file, bam_file, out)
        call(cmd.split())
    return out

In [0]:
dv['create_parallel_bams'] = create_parallel_bams

In [0]:
jobs = []
args = []
for bam in bam_files:
    for bed in beds:
        a = [samtools, bam, bed]
        args.append(a)

In [0]:
len(bam_files)*len(beds)

In [0]:
jobs = dv.map_async(create_parallel_bams, args)

In [0]:
jobs.progress

In [0]:
pwd

In [0]:
def check_bams(args):
    samtools, bam = args
    os.chdir("/home/cfriedline/eckertlab/Mitra/mapping/split_parallel/collapsed/work")
    cmd = "%s quickcheck %s" % (samtools, bam)
    return call(cmd.split())

In [0]:
dv['check_bams'] = check_bams

In [0]:
par_bams = !ls *.bam.* | grep -v '.bai$'

In [0]:
len(par_bams)

In [0]:
args = []
for b in par_bams:
    args.append((samtools, b))

In [0]:
jobs = lv.map_async(check_bams, args)

In [0]:
for j in jobs:
    if not j == 0:
        print(j)

In [0]:
job_map = {}
for b in par_bams:
    num = b.split(".")[-1]
    if not num in job_map:
        job_map[num] = []
    job_map[num].append(b)

In [0]:
for num in job_map:
    job_map[num] = sorted(job_map[num])

In [0]:
snp_args = []
for num in job_map:
    bam_files = job_map[num]
    bed_file = "contig.bed.%s" % num
    a = (samtools,
         bed_file,
         assembly, 
         bam_files, 
         bcftools, 
         "samtools_1.3.vcf.gz.%s" % num, 
         'samtools1.3')
    snp_args.append(a)

In [0]:
def call_snps(args):
    import socket, os, stopwatch
    #print(socket.gethostname())
    timer = stopwatch.Timer()
    samtools, bed, reference, bam_sorted, bcftools, raw_vcf, out_dir = args 
    if not out_dir:
        out_dir = os.environ['TMPDIR']
    raw_vcf = os.path.join(out_dir, os.path.basename(raw_vcf))
    pileup = "%s mpileup -t DP,AD,ADF,ADR,SP,INFO/AD,INFO/ADF,INFO/ADR -Iugf %s %s | %s call -f GP,GQ -vmO z -o %s" % (samtools, 
                                                                     reference, 
                                                                     ' '.join(bam_sorted), 
                                                                     bcftools,                                                                
                                                                     raw_vcf) 
    
    #print(pileup)
    #!$pileup
    timer.stop()
    return pileup, timer.elapsed

In [0]:
cmds = []
for a in snp_args:
    cmds.append(call_snps(a)[0])

In [0]:
with open("jobs.sh", "w") as j:
    j.write("#!/bin/bash\n")
    for i, a in enumerate(snp_args):
        scr = "run%d.sh" % i
        j.write("qsub %s\n" % scr)
        with open(scr, "w") as o:
            
            header = """#!/bin/bash
#$ -N psnp%d
#$ -cwd
#$ -V
#$ -o samtools1.3/psnp%d.out
#$ -e samtools1.3/psnp%d.err
#$ -q all.q""" % (i, i, i)
            
            o.write("%s\n" % header)
            o.write("%s\n" % call_snps(a)[0])

## Run on SGE
```bash
cd /gpfs_fs/home/eckertlab/Mitra/mapping/split_parallel/collapsed/work
chmod +x *.sh
./jobs.sh
```

In [0]:
call_snps(snp_args[0])